There was a problem with the scraping of details for the first set of artists so in this script, I have rescraped. The error was that 'Becky G' was inputed in every other column.

In [1]:
import pandas as pd
import wptools as wp
import json
import numpy as np
#from timeit import default_timer as timer

In [2]:
# read csv
music = pd.read_csv('track artist audio features cleaned blessing.csv')

# drop duplicate index column
music.drop('Unnamed: 0', axis=1, inplace=True)  

# isolate artist id and name
music= music[['artist_id', 'artist_name']]

# drop duplicate artist
music.drop_duplicates(inplace=True)
music

,artist_id,artist_name
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift
1,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars
3,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran
6,272ygo5fv9cYsiCLcV0HV2,Bill Conti
...,...,...
7650,5Z8mapYkacgBN46TkH9L3M,Banco De Gaia
7654,5jtGuhEEDh07yaFfm8qHg7,Cecil Taylor
7655,4rhwll8EPEHOW8diluKlrZ,Glad
7663,0n2Uel3CvQrmMsYLEfpO3s,Nektar


In [4]:
# split the data into groups because wiki has a time limit for scraping
# this is the error for time limit: (28, 'Failed to connect to en.wikipedia.org port 443 after 21022 ms: Timed out')
# 856 artists
music_first = music[:800].copy()

# 800 artists
music_second = music[800:1600].copy()  

# 642 artists
music_third = music[1601:2400].copy()

# 642 artists
music_fourth = music[2400:].copy()

,artist_id,artist_name
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift
1,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars
3,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran
6,272ygo5fv9cYsiCLcV0HV2,Bill Conti
...,...,...
1596,2ysnwxxNtSgbb9t1m2Ur4j,George Ezra
1602,3kVUvbeRdcrqQ3oHk5hPdx,Grouplove
1603,0T1KC0OHfbRO0O5bNH2tek,John Mark McMillan
1607,4ogvuDRerGhZfSf7TtzHlr,Israel Kamakawiwo'ole


In [3]:
# error: same artists have different ids
# this shouldnt be the case cos it means that same artistes have different id
music['artist_id'].nunique(), music['artist_name'].nunique()

(3235, 3207)

In [6]:
# function for scraping singer_details and saving to a df
def scrape_infobox(df_name):
    '''
    This function scrapes the infobox of artists from wikipedia and 
    appends the data as new columns to the dataframe
    '''
    count = 0 
    for index, each_name in zip(df_name.index, df_name['artist_name']):
        count += 1
        print(str(count) + ": " + each_name)
        try:
            page = wp.page(each_name).get_parse()
            keys = ['name', 'members', 'current_members', 'birth_name', 'origin', 'birth_place', 'birth_date']
            if keys[0] in page.data['infobox'].keys():  # artist name
                df_name.loc[index, 'name']= page.data['infobox'][keys[0]]
            else:
                df_name.loc[index, 'name'] = np.nan

            if keys[1] in page.data['infobox'].keys():  # bands members name
                df_name.loc[index, 'members'] = page.data['infobox'][keys[1]]
            elif keys[2] in page.data['infobox'].keys():
                df_name.loc[index, 'members'] = page.data['infobox'][keys[2]]
            else:
                df_name.loc[index, 'members'] = np.nan

            if keys[1] in page.data['infobox'].keys():  # members for bands
                df_name.loc[index, 'band'] = 'True'
            elif keys[2] in page.data['infobox'].keys():
                df_name.loc[index, 'band'] = 'True'
            else:
                df_name.loc[index, 'band'] = 'False'

            if keys[3] in page.data['infobox'].keys():  # artist birth name
                df_name.loc[index, 'birth_name'] = page.data['infobox'][keys[3]]
            else:
                df_name.loc[index, 'birth_name'] = np.nan

            if keys[4] in page.data['infobox'].keys():  # nationality
                df_name.loc[index, 'birth_place'] = page.data['infobox'][keys[4]]
            elif keys[5] in page.data['infobox'].keys():
                df_name.loc[index, 'birth_place'] = page.data['infobox'][keys[5]]
            else:
                df_name.loc[index, 'birth_place'] = np.nan

            if keys[6] in page.data['infobox'].keys():  # age
                df_name.loc[index, 'birth_date'] = page.data['infobox'][keys[6]]
            else:
                df_name.loc[index, 'birth_date'] = np.nan

        except AttributeError as err:
            pass
        except TypeError as err:
            pass
        except LookupError as err1:
            pass
        except ValueError as err2:
            pass   

    return

In [ ]:
# scrape infobox for the first music artists
music_first = scrape_infobox(df_name = music_first)

In [9]:
# check that the previous error does not occur again
music_first[music_first['name'] == 'Becky G']

,artist_id,artist_name,name,members,band,birth_name,birth_place,birth_date
71,4obzFoKoKRHIphyHzJ35G3,Becky G,Becky G,NaN,False,"Re bb eca Marie Gomez {{r|latimes|""21under21""}}","[[Inglewood, California]], U.S.","{{birth date and age|1997|03|02}} {{r|""21under..."


In [10]:
# save to a csv
music_first.to_csv('music_first_first.csv', index=False)

I got the file below from Amily. The other set of artists have been appended and now we have all the artists together.

In [7]:
artist = pd.read_csv('artist_details_unique_artist_name_2.csv')
artist

,artist_id,artist_name,name,members,band,birth_name,birth_place,birth_date
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,Taylor Swift,NaN,False,Taylor Alison Swift,"[[West Reading, Pennsylvania]], U.S.",{{Birth date and age|1989|12|13}}
1,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett,Thomas Rhett,NaN,False,Thomas Rhett Akins Jr.,"[[Nashville, Tennessee]], U.S.",{{birth date and age|1990|03|30}}
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,Bruno Mars,NaN,False,Peter Gene Hernandez,"[[Honolulu|Honolulu, Hawaii]], U.S.",{{birth date and age|1985|10|08}}
3,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,Ed Sheeran,NaN,False,Edward Christopher Sheeran,"[[Halifax, West Yorkshire]], England",{{Birth date and age|df|=|y|1991|2|17}}
4,272ygo5fv9cYsiCLcV0HV2,Bill Conti,Bill Conti,NaN,False,William Conti,"[[Providence, Rhode Island]], U.S.",{{Birth date and age|1942|4|13}}
...,...,...,...,...,...,...,...,...
3202,5Z8mapYkacgBN46TkH9L3M,Banco De Gaia,Banco de Gaia,Toby Marks,True,NaN,England,NaN
3203,5jtGuhEEDh07yaFfm8qHg7,Cecil Taylor,Cecil Taylor,NaN,False,Cecil Percival Taylor,"[[Long Island City]], New York, U.S.",{{Birth date|1929|3|25}}
3204,4rhwll8EPEHOW8diluKlrZ,Glad,NaN,NaN,NaN,NaN,NaN,NaN
3205,0n2Uel3CvQrmMsYLEfpO3s,Nektar,Nektar,{{plainlist|\n'''Nektar''' (US-based band)<br>...,True,NaN,"[[Hamburg]], West Germany",NaN


In [3]:
# read the gender data downloaded from Kaggle
gender_df = pd.read_csv('singers_gender.csv', encoding='latin1')
gender_df

,artist,gender,category
0,12 Gauge,male,African-American_male_rappers
1,1987,male,Swedish_male_singers
2,2 Chainz,male,African-American_male_rappers
3,2 Pistols,male,African-American_male_rappers
4,21 Savage,male,African-American_male_rappers
...,...,...,...
23172,Øystein Sunde,male,Male_guitarists
23173,Øyvind Nypan,male,Male_guitarists
23174,Úna MacLochlainn,female,Irish_female_singers
23175,Úna Palliser,female,Irish_female_singers


In [15]:
# combine with dataset of all artists infobox
artists_detail_spotify_name_gender = artist.merge(gender_df, how='left', left_on='artist_name', right_on='artist')
artists_detail_spotify_name_gender

,artist_id,artist_name,name,members,band,birth_name,birth_place,birth_date,artist,gender,category
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,Taylor Swift,NaN,False,Taylor Alison Swift,"[[West Reading, Pennsylvania]], U.S.",{{Birth date and age|1989|12|13}},Taylor Swift,female,American_female_country_singers
1,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett,Thomas Rhett,NaN,False,Thomas Rhett Akins Jr.,"[[Nashville, Tennessee]], U.S.",{{birth date and age|1990|03|30}},Thomas Rhett,male,American_male_singers
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,Bruno Mars,NaN,False,Peter Gene Hernandez,"[[Honolulu|Honolulu, Hawaii]], U.S.",{{birth date and age|1985|10|08}},Bruno Mars,male,American_male_singer-songwriters
3,6eUKZXaKkcviH0Ku9w2n3V,Ed Sheeran,Ed Sheeran,NaN,False,Edward Christopher Sheeran,"[[Halifax, West Yorkshire]], England",{{Birth date and age|df|=|y|1991|2|17}},Ed Sheeran,male,English_male_singers
4,272ygo5fv9cYsiCLcV0HV2,Bill Conti,Bill Conti,NaN,False,William Conti,"[[Providence, Rhode Island]], U.S.",{{Birth date and age|1942|4|13}},NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3202,5Z8mapYkacgBN46TkH9L3M,Banco De Gaia,Banco de Gaia,Toby Marks,True,NaN,England,NaN,NaN,NaN,NaN
3203,5jtGuhEEDh07yaFfm8qHg7,Cecil Taylor,Cecil Taylor,NaN,False,Cecil Percival Taylor,"[[Long Island City]], New York, U.S.",{{Birth date|1929|3|25}},NaN,NaN,NaN
3204,4rhwll8EPEHOW8diluKlrZ,Glad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3205,0n2Uel3CvQrmMsYLEfpO3s,Nektar,Nektar,{{plainlist|\n'''Nektar''' (US-based band)<br>...,True,NaN,"[[Hamburg]], West Germany",NaN,NaN,NaN,NaN


In [16]:
# save the df to csv
artists_detail_spotify_name_gender.to_csv('artists_detail_spotify_name_gender_2.csv', index=False)